In [1]:
print("test")

test


In [3]:
import numpy as np
import pandas as pd
from scipy.signal import welch
from scipy.spatial.distance import euclidean
from scipy.fft import fft
import matplotlib.pyplot as plt

# Load data
df = pd.read_csv("../data/eye_movement_type/combined_data.csv")

# Columns of interest
gaze_cols = ['Gaze point X', 'Gaze point Y']
movement_cols = ['Eye movement type', 'Gaze event duration']
pupil_cols = ['Pupil diameter left', 'Pupil diameter right']
user_col = 'Participant name'

# Preprocessing: Handle missing values
df = df.dropna(subset=gaze_cols + movement_cols + pupil_cols)

# Normalize gaze points
df['Gaze point X norm'] = df['Gaze point X'] / df['Recording resolution width']
df['Gaze point Y norm'] = df['Gaze point Y'] / df['Recording resolution height']

# Time-Domain Features
time_features = df.groupby(user_col).agg({
    'Gaze point X': ['mean', 'var', 'std'],
    'Gaze point Y': ['mean', 'var', 'std'],
    'Gaze event duration': ['mean', 'var', 'std']
}).reset_index()
time_features.columns = ['_'.join(col).strip('_') for col in time_features.columns]

# Frequency-Domain Features
def compute_frequency_features(signal):
    freqs, psd = welch(signal, nperseg=min(256, len(signal)))
    dominant_freq = freqs[np.argmax(psd)]
    return pd.Series({'psd_mean': psd.mean(), 'dominant_freq': dominant_freq})

freq_features = df.groupby(user_col).apply(
    lambda group: compute_frequency_features(group['Gaze point X'])
).reset_index()

# Spatial Features
def compute_spatial_features(group):
    distances = [euclidean((x1, y1), (x2, y2)) for (x1, y1), (x2, y2) in zip(zip(group['Gaze point X'], group['Gaze point Y']), zip(group['Gaze point X'][1:], group['Gaze point Y'][1:]))]
    angles = np.arctan2(np.diff(group['Gaze point Y']), np.diff(group['Gaze point X']))
    return pd.Series({
        'mean_distance': np.mean(distances),
        'std_distance': np.std(distances),
        'mean_angle': np.mean(angles),
        'std_angle': np.std(angles)
    })

spatial_features = df.groupby(user_col).apply(compute_spatial_features).reset_index()

# Behavioral Features
behavioral_features = df.groupby(user_col).agg({
    'Eye movement type': lambda x: x.value_counts(normalize=True).to_dict(),
    'Pupil diameter left': ['mean', 'std'],
    'Pupil diameter right': ['mean', 'std']
}).reset_index()
behavioral_features.columns = ['_'.join(col).strip('_') for col in behavioral_features.columns]

# Combine all features
all_features = pd.merge(time_features, freq_features, on=user_col)
all_features = pd.merge(all_features, spatial_features, on=user_col)
all_features = pd.merge(all_features, behavioral_features, on=user_col)

# Save new DataFrame
all_features.to_csv("features_for_authentication.csv", index=False)
print("New DataFrame with features saved as 'features_for_authentication.csv'.")

# Optional: Visualize Heatmap of Gaze Points
plt.hist2d(df['Gaze point X'], df['Gaze point Y'], bins=(50, 50), cmap="Reds")
plt.colorbar()
plt.title("Gaze Points Heatmap")
plt.xlabel("Gaze point X")
plt.ylabel("Gaze point Y")
plt.show()


/tmp/ipykernel_4318/1132620525.py:38: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  freq_features = df.groupby(user_col).apply(
/tmp/ipykernel_4318/1132620525.py:53: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  spatial_features = df.groupby(user_col).apply(compute_spatial_features).reset_index()


TypeError: agg function failed [how->mean,dtype->object]